In [ ]:
"""
Test script for validating the behavior of the Streamlit app defined in 'Home.py'.

This test uses the Streamlit testing API to:
- Load the app from the 'Home.py' file.
- Set a mock secret value for the key 'WORD'.
- Run the app and ensure no unhandled exceptions occur.
- If a warning is raised during execution, verify that its message is 'Try again.'.

Assertions:
- No exception should be thrown during execution.
- If any warnings are present, the first warning must have the value 'Try again.'.
"""

from streamlit.testing.v1 import AppTest

at = AppTest.from_file("Home.py")

at.secrets["WORD"] = "Foobar"

at.run()

assert not at.exception

if at.warning:
    assert at.warning[0].value == "Try again."

2025-06-10 12:19:47.385 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
"Same Code with other import style"

from streamlit.testing.v1 import AppTest

at = AppTest. from_file("Home.py")

at. secrets ["WORD"] = "Foobar"

at.run()

assert not at.exception

2025-06-10 13:16:58.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
"""
Test runner for the MacroNetwork class using the TestNetwork test suite.

This script:
- Ensures the project root is in sys.path for imports.
- Imports the TestNetwork class from test_macro_network.
- Patches the setup_method of TestNetwork to use a specific data path for MacroNetwork.
- Defines a list of test methods to run.
- Instantiates TestNetwork, runs each test, and collects results.
- Prints a summary of test outcomes (PASSED, FAILED, or ERROR).

Usage:
- Place this script in your test directory.
- Adjust 'data_path' if your mart directory is elsewhere.
- Run the script in a Jupyter cell or as a standalone Python script.

Note:
- Ensure that test_macro_network.py and MacroNetwork are correctly implemented and available.
"""


import sys
import os

# Add your project root to sys.path if needed
project_root = "/home/learner/Desktop/internship/Dealy_analysis/tempocom-app"
if project_root not in sys.path:
    sys.path.append(project_root)

from test_macro_network import TestNetwork

# Set the correct data path for MacroNetwork
data_path = "/home/learner/Desktop/internship/Dealy_analysis/tempocom-app/mart"  # adjust if needed

# Patch setup_method to use the correct data path
def setup_method(self):
    print("Setting up test network...")
    self.network = MacroNetwork(path_to_mart=data_path)
    print("Test network setup complete")
TestNetwork.setup_method = setup_method

# List of test methods to run
test_methods = [
    "test_get_shortest_path",
    "test_get_shortest_path_no_path",
    "test_get_shortest_path_same_station",
    "test_cut_and_get_shortest_path"
]

# Run tests and collect results
test_instance = TestNetwork()
results = []
test_instance.setup_method()
for method in test_methods:
    try:
        getattr(test_instance, method)()
        print(f"{method}: PASSED")
        results.append((method, "PASSED"))
    except AssertionError as e:
        print(f"{method}: FAILED\n  AssertionError: {e}")
        results.append((method, f"FAILED: {e}"))
    except Exception as e:
        print(f"{method}: ERROR\n  {type(e).__name__}: {e}")
        results.append((method, f"ERROR: {e}"))

# Print summary
print("\nTest Summary:")
for method, result in results:
    print(f"{method}: {result}")

Setting up test network...
Test network setup complete
test_get_shortest_path: PASSED
test_get_shortest_path_no_path: PASSED
test_get_shortest_path_same_station: PASSED
test_cut_and_get_shortest_path: PASSED

Test Summary:
test_get_shortest_path: PASSED
test_get_shortest_path_no_path: PASSED
test_get_shortest_path_same_station: PASSED
test_cut_and_get_shortest_path: PASSED


In [ ]:
"""
Unit tests for the DelayBubbleMap class in pages.Delay_network.

This script:
- Creates mock station and delay data as temporary CSV files.
- Instantiates DelayBubbleMap with the mock data.
- Tests that the merged data is correct and contains expected columns and types.
- Verifies that the total delay calculation is accurate.
- Checks that render_map returns a folium.Map object.

Usage:
- Run this script in a Jupyter cell or as a standalone Python script.
- Requires pandas, folium, and DelayBubbleMap implementation.

Note:
- If pytest is not installed, a fallback for pytest.approx is provided.
"""

import os
import sys

# Add your project root to sys.path if needed
project_root = "/home/learner/Desktop/internship/Dealy_analysis/tempocom-app"
if project_root not in sys.path:
    sys.path.append(project_root)

from pages.Delay_network import DelayBubbleMap  # Adjust if you moved the file

import tempfile

def create_mock_station_data():
    data = {
        'Name_FR': ['Bruxelles-Central', 'Charleroi-Central'],
        'Geo_Point': ['(50.8467, 4.3525)', '(50.4113, 4.4447)']
    }
    df = pd.DataFrame(data)
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
    df.to_csv(tmp.name, index=False)
    return tmp.name

def create_mock_delay_data():
    data = {
        'Stopping place (FR)': ['Bruxelles-Central', 'Charleroi-Central', 'Charleroi-Central'],
        'Delay at arrival': [300, 120, 240]  # in seconds
    }
    df = pd.DataFrame(data)
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
    df.to_csv(tmp.name, index=False)
    return tmp.name

def test_prepare_data():
    stations_path = create_mock_station_data()
    delay_data_path = create_mock_delay_data()
    bubble_map = DelayBubbleMap(stations_path=stations_path, delay_data_path=delay_data_path)

    # Check merged data
    merged = bubble_map.merged
    assert not merged.empty
    assert 'Geo_Point' in merged.columns
    assert all(isinstance(pt, tuple) for pt in merged['Geo_Point'])
    assert merged.loc[merged['Name_FR'] == 'Charleroi-Central', 'Total_Delay_Minutes'].iloc[0] == pytest.approx((120 + 240) / 60)
    print("test_prepare_data: PASSED")

def test_render_map_returns_folium_map():
    stations_path = create_mock_station_data()
    delay_data_path = create_mock_delay_data()
    bubble_map = DelayBubbleMap(stations_path=stations_path, delay_data_path=delay_data_path)
    fmap = bubble_map.render_map()

    import folium
    assert isinstance(fmap, folium.Map)
    print("test_render_map_returns_folium_map: PASSED")

# Run tests manually
try:
    import pytest
except ImportError:
    # fallback for pytest.approx
    class pytest:
        @staticmethod
        def approx(val):
            return val

test_prepare_data()
test_render_map_returns_folium_map()

test_prepare_data: PASSED
test_render_map_returns_folium_map: PASSED
